# Dataset Generation Script

This script performs the following operations:
1. Extracts 5000 training questions and 100 testing questions for each domain (coding, math, trivia).
2. Saves the selected datasets.
3. Generates responses using 4 different teacher models via OpenRouter.
4. Applies templates in half of the cases.
5. Saves the generated data.

In [1]:
import os
import json
import random
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
# from tqdm.notebook import tqdm # Causing ImportError with IProgress
from tqdm import tqdm # Use standard tqdm instead
from openai import OpenAI

# --- Configuration ---

# Ensure you have your OpenRouter API key set in your environment
# os.environ["OPENROUTER_API_KEY"] = "sk-or-v1-..."
API_KEY = os.getenv("OPENROUTER_API_KEY")

if not API_KEY:
    raise ValueError("Please set OPENROUTER_API_KEY environment variable.")

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=API_KEY,
)

DOMAINS = ["trivia", "math", "general"]
TEACHER_MODELS = [
    "qwen/qwen-2.5-72b-instruct",
    "qwen/qwen-2.5-7b-instruct",
    "meta-llama/llama-3.1-70b-instruct",
    "meta-llama/llama-3.1-8b-instruct",
]
TRAIN_SIZE = 5000
TEST_SIZE = 100
SEED = 42
MAX_WORKERS = 5 # reduced per-config workers since we run configs in parallel
MAX_PARALLEL_CONFIGS = 4 # Number of configurations to run simultaneously


In [2]:
from datasets import load_dataset
import json

/Users/jrwong/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
try:
    print("Testing OpenRouter connection...")
    completion = client.chat.completions.create(
        model="meta-llama/llama-3.1-8b-instruct",
        messages=[
            {"role": "user", "content": "Hello, world!"}
        ]
    )
    print("Connection successful!")
    print("Response:", completion.choices[0].message.content)
except Exception as e:
    print("Connection failed:", str(e))
    raise e

Testing OpenRouter connection...
Connection successful!
Response: Hello! It's nice to meet you! How can I assist you today?


In [4]:
def load_and_select_data(domain, split, count):
    """
    Loads data for a domain/split.
    If 'selected' file exists, returns it.
    Otherwise, samples from 'raw' file, saves to 'selected', and returns it.
    """
    base_path = os.path.join("experiments", domain, "questions")
    selected_file = os.path.join(base_path, f"{split}_selected.jsonl")
    raw_file = os.path.join(base_path, f"{split}_raw.jsonl")

    # Check if selected file exists
    if os.path.exists(selected_file):
        print(f"Loading existing selected data for {domain}/{split}...")
        with open(selected_file, 'r', encoding='utf-8') as f:
            data = [json.loads(line) for line in f]
        # Verify size (warn if mismatch, but allow reuse)
        if len(data) != count:
            print(f"Warning: Existing file has {len(data)} items, expected {count}.")
        return data

    print(f"Sampling new data for {domain}/{split}...")
    # Load raw data
    data = []
    with open(raw_file, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))

    # Sample
    random.seed(SEED)
    if len(data) > count:
        selected_data = random.sample(data, count)
    else:
        selected_data = data

    # Save selected
    with open(selected_file, 'w', encoding='utf-8') as f:
        for item in selected_data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    return selected_data

def load_template(domain):
    path = os.path.join("experiments", domain, "template.txt")
    with open(path, 'r', encoding='utf-8') as f:
        return f.read().strip()

In [5]:
def generate_single_response(item, model, template_text, use_template):
    """
    Generates a response for a single item.
    Returns the enriched item with 'response', 'model', 'template_applied'.
    """
    question = item['question']

    if use_template and template_text:
        # Assume template has [QUESTION] placeholder based on file inspection
        prompt = template_text.replace("[QUESTION]", question)
    else:
        prompt = question

    # Retry logic
    max_retries = 3
    for attempt in range(max_retries):
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "user", "content": prompt}
                ]
            )
            response = completion.choices[0].message.content

            result = item.copy()
            result['response'] = response
            result['model'] = model
            result['template_applied'] = use_template
            result['prompt'] = prompt
            return result

        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed to generate for prompt: {prompt[:50]}... Error: {e}")
                result = item.copy()
                result['error'] = str(e)
                return result
            time.sleep(2 ** attempt)  # Exponential backoff

In [6]:
def run_generation_process(domain, split, data, model, use_template):
    """
    Runs generation for a specific configuration and saves results.
    """
    template_text = load_template(domain)

    # Construct output filename
    # Format: experiments/{domain}/{model_clean}/{split}_{template}.jsonl
    model_clean = model.replace("/", "_")
    template_str = "with_template" if use_template else "no_template"
    
    # Create model directory if it doesn't exist
    model_dir = os.path.join("experiments", domain, "dataset", model_clean)
    os.makedirs(model_dir, exist_ok=True)

    output_filename = f"{split}_{template_str}.jsonl"
    output_path = os.path.join(model_dir, output_filename)

    if os.path.exists(output_path):
        print(f"  [Skipping] {output_filename} already exists.")
        return

    print(f"  [Generating] {domain} | {split} | {model} | Template={use_template}")

    results = []
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = [
            executor.submit(generate_single_response, item, model, template_text, use_template)
            for item in data
        ]

        # Using tqdm for progress within the generation
        for future in tqdm(as_completed(futures), total=len(data), desc=f"    Progress", leave=False):
            results.append(future.result())

    # Save results
    with open(output_path, 'w', encoding='utf-8') as f:
        for res in results:
            f.write(json.dumps(res, ensure_ascii=False) + "\n")
    print(f"  [Saved] {output_filename}")

In [7]:
# --- Main Execution Loop ---

# Pre-load all data serially to avoid race conditions
domain_data = {}
for domain in DOMAINS:
    print(f"Loading data for {domain}...")
    try:
        train_data = load_and_select_data(domain, "train", TRAIN_SIZE)
        test_data = load_and_select_data(domain, "test", TEST_SIZE)
        domain_data[domain] = {"train": train_data, "test": test_data}
    except Exception as e:
        print(f"Skipping domain {domain} due to error: {e}")

# Collect all tasks
tasks = []
for domain, datasets in domain_data.items():
    for model in TEACHER_MODELS:
        for use_template in [True, False]:
            for split_name in ["train", "test"]:
                tasks.append({
                    "domain": domain,
                    "split": split_name,
                    "data": datasets[split_name],
                    "model": model,
                    "use_template": use_template
                })

print(f"\nTotal configurations to run: {len(tasks)}")

def process_task(task):
    try:
        run_generation_process(
            domain=task["domain"],
            split=task["split"],
            data=task["data"],
            model=task["model"],
            use_template=task["use_template"]
        )
        return f"Success: {task['domain']} {task['model']}"
    except Exception as e:
        return f"Error in {task['domain']} {task['model']}: {e}"

print("Starting parallel execution...")
with ThreadPoolExecutor(max_workers=MAX_PARALLEL_CONFIGS) as executor:
    futures = [executor.submit(process_task, task) for task in tasks]

    for future in as_completed(futures):
        print(future.result())

print("\nAll generation tasks completed!")

Loading data for trivia...
Loading existing selected data for trivia/train...
Loading existing selected data for trivia/test...
Loading data for math...
Loading existing selected data for math/train...
Loading existing selected data for math/test...
Loading data for general-elif...
Loading existing selected data for general-elif/train...
Loading existing selected data for general-elif/test...

Total configurations to run: 48
Starting parallel execution...
  [Skipping] train_generated_qwen_qwen-2.5-72b-instruct_with_template.jsonl already exists.
  [Skipping] train_generated_qwen_qwen-2.5-72b-instruct_no_template.jsonl already exists.
Success: trivia qwen/qwen-2.5-72b-instruct
Success: trivia qwen/qwen-2.5-72b-instruct
  [Skipping] test_generated_qwen_qwen-2.5-72b-instruct_with_template.jsonl already exists.
Success: trivia qwen/qwen-2.5-72b-instruct
  [Skipping] test_generated_qwen_qwen-2.5-72b-instruct_no_template.jsonl already exists.
Success: trivia qwen/qwen-2.5-72b-instruct
  [Ski




    Progress:   0%|          | 2/5000 [00:03<2:41:13,  1.94s/it]

    Progress:   0%|          | 4/5000 [00:05<1:19:18,  1.05it/s]





    Progress:   0%|          | 5/5000 [00:07<2:08:12,  1.54s/it]






    Progress:   0%|          | 9/5000 [00:12<1:32:20,  1.11s/it]


    Progress:   0%|          | 10/5000 [00:14<1:47:12,  1.29s/it]




    Progress:   0%|          | 13/5000 [00:18<1:58:38,  1.43s/it]



    Progress:   0%|          | 15/5000 [00:22<2:16:42,  1.65s/it]

    Progress:   0%|          | 16/5000 [00:22<1:42:49,  1.24s/it]




    Progress:   0%|          | 17/5000 [00:25<2:27:57,  1.78s/it]

    Progress:   0%|          | 20/5000 [00:26<1:04:51,  1.28it/s]









    Progress:   0%|          | 21/5000 [00:31<2:59:03,  2.16s/it]

    Progress:   0%|          | 22/5000 [00:34<3:09:01,  2.28s/it]

    Progress:   0%|          | 23/5000 [00:35<2:39:02,  1.92s/it]



    Progress:   0%|          | 24/5000 [00:37<2:51:16,  2.07s/it]









    Progress:   0%|        

  [Saved] train_generated_meta-llama_llama-3.1-70b-instruct_no_template.jsonl
  [Generating] math | test | meta-llama/llama-3.1-70b-instruct | Template=False



    Progress:  82%|████████▏ | 4121/5000 [1:58:39<12:18,  1.19it/s]




    Progress:  82%|████████▏ | 4123/5000 [1:58:43<19:34,  1.34s/it]





    Progress:  82%|████████▎ | 4125/5000 [1:58:44<13:11,  1.10it/s]

    Progress:  83%|████████▎ | 4126/5000 [1:58:45<13:01,  1.12it/s]



    Progress:  83%|████████▎ | 4128/5000 [1:58:46<11:38,  1.25it/s]


    Progress:  83%|████████▎ | 4129/5000 [1:58:48<17:02,  1.17s/it]



    Progress:  83%|████████▎ | 4130/5000 [1:58:50<19:19,  1.33s/it]





    Progress:  83%|████████▎ | 4132/5000 [1:58:54<21:49,  1.51s/it]




    Progress:  83%|████████▎ | 4133/5000 [1:58:56<25:24,  1.76s/it]

    Progress:  83%|████████▎ | 4135/5000 [1:58:57<17:05,  1.19s/it]

    Progress:  83%|████████▎ | 4137/5000 [1:59:00<17:27,  1.21s/it]


    Progress:  83%|████████▎ | 4138/5000 [1:59:02<19:27,  1.35s/it]








    Progress:  83%|████████▎ | 4139/5000 [1:59:05<30:01,  2.09s/it]





    Progress:  83%|████████▎ | 4140/5000 [1:59:10<41:07,  2.87s/it]


 

  [Saved] test_generated_meta-llama_llama-3.1-70b-instruct_no_template.jsonl
  [Generating] math | train | meta-llama/llama-3.1-8b-instruct | Template=True


...

  [Saved] test_generated_meta-llama_llama-3.1-8b-instruct_no_template.jsonl


  [Saved] train_generated_qwen_qwen-2.5-72b-instruct_with_template.jsonl


  [Saved] train_generated_meta-llama_llama-3.1-8b-instruct_with_template.jsonl


  [Saved] train_generated_meta-llama_llama-3.1-8b-instruct_no_template.jsonl


KeyboardInterrupt: 